# New York AirBnB

__Import statements__

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    FunctionTransformer
)
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

### Importing the dataset

In [2]:
data = pd.read_csv("../data/raw/airbnb.csv")

/home/utkarshsaboo45/miniconda3/envs/env_airbnb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,150,30,48,2019-11-04,0.33,3,322,0,NaN
1,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...",4869,LisaRoxanne,Brooklyn,Bedford-Stuyvesant,40.68494,-73.95765,Entire home/apt,73,1,408,2021-06-29,4.91,1,220,38,NaN
2,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,30,50,2016-06-05,0.53,2,365,0,NaN
3,5136,"Spacious Brooklyn Duplex, Patio + Garden",7378,Rebecca,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,275,5,2,2021-08-08,0.02,1,91,1,NaN
4,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Midtown,40.76457,-73.98317,Private room,68,2,505,2021-10-20,3.70,1,218,31,NaN


In [4]:
# Drop license, id, last_review
# data.drop(columns=["id", "license", "last_review"], inplace=True)

In [5]:
data.dropna(subset=["reviews_per_month"], inplace=True)

In [6]:
data["year"] = pd.DatetimeIndex(data["last_review"]).year
data["month"] = pd.DatetimeIndex(data["last_review"]).month
data["date"] = pd.DatetimeIndex(data["last_review"]).day

In [7]:
# Splitting the data

In [8]:
train_big_data, test_data = train_test_split(data, test_size=0.2, random_state=123)
train_data, val_data = train_test_split(train_big_data, test_size=0.25, random_state=123)

In [9]:
## Some Stupidity

# def find_no_of_listings(data):
#     temp_df = pd.DataFrame(data["host_id"].value_counts()).reset_index()

#     temp_df.rename(columns={
#         "index": "host_id",
#         "host_id": "number_of_listings"
#     }, inplace=True)

#     data = pd.merge(data, temp_df, on="host_id")

#     data["number_of_listings"] = pd.cut(
#         data.number_of_listings,
#         bins=[0, 1, 10, np.inf],
#         labels=["1", "2-10", ">10"]
#     )

#     return data

In [10]:
# find_no_of_listings(train_data)["number_of_listings"].value_counts()

In [11]:
# find_no_of_listings(test_data)["number_of_listings"].value_counts()

In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17100 entries, 19360 to 13967
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              17100 non-null  int64  
 1   name                            17098 non-null  object 
 2   host_id                         17100 non-null  int64  
 3   host_name                       17094 non-null  object 
 4   neighbourhood_group             17100 non-null  object 
 5   neighbourhood                   17100 non-null  object 
 6   latitude                        17100 non-null  float64
 7   longitude                       17100 non-null  float64
 8   room_type                       17100 non-null  object 
 9   price                           17100 non-null  int64  
 10  minimum_nights                  17100 non-null  int64  
 11  number_of_reviews               17100 non-null  int64  
 12  last_review                 

In [13]:
data.head().T


,0,1,2,3,4
id,2595,3831,5121,5136,5178
name,Skylit Midtown Castle,"Whole flr w/private bdrm, bath & kitchen(pls r...",BlissArtsSpace!,"Spacious Brooklyn Duplex, Patio + Garden",Large Furnished Room Near B'way
host_id,2845,4869,7356,7378,8967
host_name,Jennifer,LisaRoxanne,Garon,Rebecca,Shunichi
neighbourhood_group,Manhattan,Brooklyn,Brooklyn,Brooklyn,Manhattan
neighbourhood,Midtown,Bedford-Stuyvesant,Bedford-Stuyvesant,Sunset Park,Midtown
latitude,40.75356,40.68494,40.68535,40.66265,40.76457
longitude,-73.98559,-73.95765,-73.95512,-73.99454,-73.98317
room_type,Entire home/apt,Entire home/apt,Private room,Entire home/apt,Private room
price,150,73,60,275,68


In [14]:
# find_no_of_listings(data)[["number_of_listings"]].value_counts()

In [15]:
# pd.DataFrame(data["neighbourhood"].value_counts()).to_csv("temp.csv")

In [16]:
# data["number_of_listings"] =

In [17]:
numeric_features = [
    "latitude",
    "longitude",
    "price",
    "minimum_nights",
    "calculated_host_listings_count",
    "availability_365",
    "year",
    "month",
    "date"
]

categorical_features = [
    "neighbourhood_group",
    "neighbourhood",
    "room_type"
]

text_features = [
    "name"
]

drop_features = [
    "id",
    "license",
    "last_review",
    "host_id",
    "host_name",
    "number_of_reviews",
    "number_of_reviews_ltm"
]

passthrough_features = []

target_column = "reviews_per_month"

assert (
    len(numeric_features) +
    len(categorical_features) +
    len(text_features) +
    len(drop_features) +
    len(passthrough_features) +
    len([target_column])
) == len(train_data.columns)

In [18]:
X_train, y_train = train_data.drop(columns=[target_column]), train_data[target_column]

In [19]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17100 entries, 19360 to 13967
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              17100 non-null  int64  
 1   name                            17098 non-null  object 
 2   host_id                         17100 non-null  int64  
 3   host_name                       17094 non-null  object 
 4   neighbourhood_group             17100 non-null  object 
 5   neighbourhood                   17100 non-null  object 
 6   latitude                        17100 non-null  float64
 7   longitude                       17100 non-null  float64
 8   room_type                       17100 non-null  object 
 9   price                           17100 non-null  int64  
 10  minimum_nights                  17100 non-null  int64  
 11  number_of_reviews               17100 non-null  int64  
 12  last_review                 

In [20]:
function_transformer = FunctionTransformer(
    np.reshape, kw_args={"newshape": -1}
)

pipe_text_feats = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    function_transformer,
    CountVectorizer(stop_words="english", max_features=100)
)

column_transformer = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OneHotEncoder(sparse=False), categorical_features),
    (pipe_text_feats, text_features),
    ("drop", drop_features),
    ("passthrough", passthrough_features)
)

In [21]:
column_transformer.fit(X_train)

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['latitude', 'longitude', 'price',
                                  'minimum_nights',
                                  'calculated_host_listings_count',
                                  'availability_365', 'year', 'month',
                                  'date']),
                                ('onehotencoder', OneHotEncoder(sparse=False),
                                 ['neighbourhood_group', 'neighbourhood',
                                  'room_type']),
                                ('pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(s...uent')),
                                                 ('functiontransformer',
                                                  FunctionTransformer(func=<function reshape at 0x7fa1e80fc550>,
                                                   

In [22]:
new_columns = (
    numeric_features +
    column_transformer.named_transformers_["onehotencoder"].get_feature_names_out().tolist() +
    column_transformer.named_transformers_["pipeline"].named_steps["countvectorizer"].get_feature_names_out().tolist() +
    passthrough_features
)

In [23]:
pd.DataFrame(column_transformer.transform(X_train), columns=new_columns)

,latitude,longitude,price,minimum_nights,calculated_host_listings_count,availability_365,year,month,date,neighbourhood_group_Bronx,...,times,townhouse,train,upper,view,views,village,west,williamsburg,york
0,-0.406307,0.111532,-0.088612,0.399872,-0.259772,-0.237283,0.764204,-0.244894,-0.942082,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.960255,-0.597747,-0.051700,-0.550828,-0.259772,-0.903438,0.764204,-0.563494,1.002941,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.122256,-0.881149,-0.002484,-0.664912,5.811509,-0.266247,0.764204,0.710907,0.183984,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.575909,-0.306592,-0.113220,0.399872,-0.155693,-0.135912,0.764204,-1.200694,0.286353,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.913452,1.029390,-0.338795,-0.626884,-0.190386,0.204406,0.764204,-2.156495,-1.453930,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17095,1.229243,-0.304653,-0.379808,-0.702940,-0.259772,-0.903438,0.764204,0.710907,-0.839712,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17096,0.790320,0.746570,-0.404416,0.323816,-0.225079,-0.686214,0.764204,0.073707,0.695832,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17097,-0.617706,0.056480,-0.474139,-0.702940,0.052466,1.167433,0.764204,0.710907,0.491093,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17098,1.412290,-0.017181,-0.310085,-0.626884,-0.259772,-0.816548,-0.314539,1.348107,0.183984,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
